# Understanding Ontologies with Cognee

This notebook demonstrates how to work with ontologies in scientific research using the Cognee framework. We'll explore how ontologies can enhance our understanding and querying of scientific papers.

## What is an Ontology?

An ontology is a formal representation of knowledge that defines:
- Concepts within a domain
- Relationships between concepts
- Properties and attributes
- Rules and constraints

Key terms:
- **Classes**: Categories or types (e.g., Disease, Symptom)
- **Instances**: Specific examples of classes (e.g., Type 2 Diabetes)
- **Properties**: Relationships between classes/instances (e.g., hasSymptom)
- **Axioms**: Logical statements defining relationships

## Setup

First, let's install the required packages and set up our environment:

In [17]:
# Install required package
# !pip install cognee

In [18]:
# Import required libraries
import cognee
import asyncio
from cognee.shared.logging_utils import get_logger
import os
import textwrap
from cognee.api.v1.search import SearchType
from cognee.api.v1.visualize.visualize import visualize_graph

logger = get_logger()

# Set up OpenAI API key (required for Cognee's LLM functionality)
os.environ["LLM_API_KEY"] = "your-api-key-here"  # Replace with your API key

## Creating the Pipeline

Let's create a pipeline that will:
1. Clean existing data
2. Process scientific papers
3. Apply ontological knowledge

In [26]:
async def run_pipeline(ontology_path=None):
    # Clean existing data
    await cognee.prune.prune_data()
    await cognee.prune.prune_system(metadata=True)
    
    # Set up path to scientific papers
    scientific_papers_dir = os.path.join(
        os.path.dirname(os.path.dirname(os.path.abspath("."))), 
        "cognee",
        "examples",
        "data", 
        "scientific_papers/"
    )
    
    # Add papers to the system
    await cognee.add(scientific_papers_dir)
    
    # Cognify with optional ontology
    return await cognee.cognify(ontology_file_path=ontology_path)

async def query_pipeline(questions):
    answers = []
    for question in questions:
        search_results = await cognee.search(
            query_type=SearchType.GRAPH_COMPLETION,
            query_text=question,
        )
        answers.append(search_results)
    return answers

## Running the Demo

Let's test our system with some medical questions, comparing results with and without ontological knowledge:

In [27]:
# Test questions
questions = [
    "What are common risk factors for Type 2 Diabetes?",
    "What preventive measures reduce the risk of Hypertension?",
    "What symptoms indicate possible Cardiovascular Disease?",
    "What diseases are associated with Obesity?"
]

# Path to medical ontology
ontology_path = "examples/python/ontology_input_example/enriched_medical_ontology_with_classes.owl"  # Update with your ontology path

# Run with ontology
print("\n--- Results WITH ontology ---\n")
await run_pipeline(ontology_path=ontology_path)
answers_with = await query_pipeline(questions)
for q, a in zip(questions, answers_with):
    print(f"Q: {q}\nA: {a}\n")

In [23]:
# Run without ontology
print("\n--- Results WITHOUT ontology ---\n")
await run_pipeline()
answers_without = await query_pipeline(questions)
for q, a in zip(questions, answers_without):
    print(f"Q: {q}\nA: {a}\n")

## Visualizing the Knowledge Graph

Let's visualize how our ontology connects different medical concepts:

In [25]:
from cognee.api.v1.visualize import visualize_graph
await visualize_graph()

## Understanding the Results

The demonstration above shows how ontologies enhance our analysis by:

1. **Making Connections**: 
   - Linking related medical concepts even when not explicitly stated
   - Identifying relationships between symptoms, diseases, and risk factors

2. **Standardizing Terms**: 
   - Unifying different ways of referring to the same medical condition
   - Ensuring consistent terminology across documents

3. **Enabling Inference**: 
   - Drawing conclusions based on ontological relationships
   - Discovering implicit connections in the data

## Next Steps

To learn more about Cognee and ontologies:
1. Check out the [Cognee documentation](https://docs.cognee.ai/)
2. Explore more examples in the `examples` directory
3. Try creating your own domain-specific ontology

Remember to:
- Place your scientific papers in the appropriate directory
- Update the ontology path to point to your .owl file
- Replace the API key with your own OpenAI key